In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Installing necessary packages

In [ ]:
pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 249 kB 8.1 MB/s 
     |████████████████████████████████| 365 kB 59.0 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 9.1 MB 54.1 MB/s 
     |████████████████████████████████| 4.7 MB 48.0 MB/s 
     |████████████████████████████████| 1.3 MB 58.0 MB/s 
     |████████████████████████████████| 6.6 MB 53.3 MB/s 
     |████████████████████████████████| 1.8 MB 46.1 MB/s 
     |████████████████████████████████| 120 kB 64.9 MB/s 
     |████████████████████████████████| 181 kB 64.2 MB/s 
     |████████████████████████████████| 158 kB 63.1 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 157 kB 69.1 MB/s 
     |████████████████████████████████| 157 kB 67.7 MB/s 
     |████████████████████████████████| 157 kB 69.9 MB/s 
     |██████████████████████████████

In [ ]:
pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Importing libraries

In [ ]:
import warnings
import pandas as pd
import os
from datetime import datetime
import logging
import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)




# Loading the data

In [ ]:
data = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/generation_context_2.csv")

In [ ]:
data

,Paragraph,Id,Sentence,BART_Paragraph
0,"[first lines] Joy Stark: Okay, remember that t...",58-1-1,"Okay, remember that the garbage goes out on Tu...","['Joy Stark: Okay, remember that the garbage g..."
1,[first lines] Eddie Stark: 'Morning! Joy Stark...,62-0-6,Because we owe them 1200.,"[""[first lines] Eddie Stark: 'Morning! Joy Sta..."
2,FBI Agent Nicole Scott: You want another crool...,75-1-3,I've had two already. Who eats three croolers ...,['FBI Agent Nicole Scott: You want another cro...
3,Janey Cooper: Would you like a muffin? Jess Ma...,75-5-4,Try one.,['Janey Cooper: Would you like a muffin? Jess ...
4,Lucky: Red Airedale rules! Cadpig: Yellow's my...,107-5-2,I like the brown one. It looks like gravy.,"[""Lucky: Red Airedale rules! Cadpig: Yellow's ..."
...,...,...,...,...
295,Julie Yamamoto: They were excited to get me on...,6100-3-2,First of all I'm ranked 83! And I've only been...,['Julie Yamamoto: They were excited to get me ...
296,Eric Saiylor: Is it just your kills in Counter...,6115-1-0,Is it just your kills in Counter Strike that s...,['Eric Saiylor: Is it just your kills in Count...
297,Eric Saiylor: Is it just your kills in Counter...,6115-1-0,Is it just your kills in Counter Strike that s...,['Eric Saiylor: Is it just your kills in Count...
298,"Clayton Runnymede Endicott III: Marcy, Marcy, ...",6131-19-1,"I don't know anything about horses, but I know...",['Clayton Runnymede Endicott III: When a horse...


In [ ]:
data = data.rename(
    columns={"Paragraph" : "input_text" , "BART_Paragraph" : "target_text" }
)


data = data[['input_text', 'target_text']]

data["prefix"] = "paraphrase"

data

,input_text,target_text,prefix
0,"[first lines] Joy Stark: Okay, remember that t...","['Joy Stark: Okay, remember that the garbage g...",paraphrase
1,[first lines] Eddie Stark: 'Morning! Joy Stark...,"[""[first lines] Eddie Stark: 'Morning! Joy Sta...",paraphrase
2,FBI Agent Nicole Scott: You want another crool...,['FBI Agent Nicole Scott: You want another cro...,paraphrase
3,Janey Cooper: Would you like a muffin? Jess Ma...,['Janey Cooper: Would you like a muffin? Jess ...,paraphrase
4,Lucky: Red Airedale rules! Cadpig: Yellow's my...,"[""Lucky: Red Airedale rules! Cadpig: Yellow's ...",paraphrase
...,...,...,...
295,Julie Yamamoto: They were excited to get me on...,['Julie Yamamoto: They were excited to get me ...,paraphrase
296,Eric Saiylor: Is it just your kills in Counter...,['Eric Saiylor: Is it just your kills in Count...,paraphrase
297,Eric Saiylor: Is it just your kills in Counter...,['Eric Saiylor: Is it just your kills in Count...,paraphrase
298,"Clayton Runnymede Endicott III: Marcy, Marcy, ...",['Clayton Runnymede Endicott III: When a horse...,paraphrase


In [ ]:
train_df = data.sample(frac=0.8, random_state=25)

In [ ]:
train_df.shape

(240, 3)

In [ ]:
data2 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/full_pos.tsv', sep='\t', error_bad_lines = False)

# Data: [https://github.com/yanaiela/num_fh/blob/master/data/identification/processed/full_pos.tsv]


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 987: expected 8 fields, saw 10\nSkipping line 2052: expected 8 fields, saw 9\nSkipping line 7541: expected 8 fields, saw 9\nSkipping line 10353: expected 8 fields, saw 9\nSkipping line 11962: expected 8 fields, saw 9\nSkipping line 12492: expected 8 fields, saw 10\nSkipping line 16463: expected 8 fields, saw 9\nSkipping line 18364: expected 8 fields, saw 9\nSkipping line 20148: expected 8 fields, saw 9\nSkipping line 21767: expected 8 fields, saw 9\nSkipping line 21989: expected 8 fields, saw 9\nSkipping line 24845: expected 8 fields, saw 9\nSkipping line 25181: expected 8 fields, saw 9\nSkipping line 26118: expected 8 fields, saw 9\nSkipping line 28845: expected 8 fields, saw 9\nSkipping line 32687: expected 8 fields, saw 9\nSkipping li

In [ ]:
eval_df = data.drop(train_df.index)

# The model arguments

In [ ]:
model_args = Seq2SeqArgs()
model_args.do_sample = True
model_args.eval_batch_size = 64
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 2500
model_args.evaluate_during_training_verbose = True
model_args.fp16 = False
model_args.learning_rate = 5e-5
model_args.max_length = 128
model_args.max_seq_length = 128
model_args.num_beams = None
model_args.num_return_sequences = 3
model_args.num_train_epochs = 2
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.save_eval_checkpoints = False
model_args.save_steps = -1
model_args.top_k = 50
model_args.top_p = 0.95
model_args.train_batch_size = 1
model_args.use_multiprocessing = False

In [ ]:
#use_cuda=False

In [ ]:
import torch


if torch.cuda.is_available():
	dev = "cuda:0"
else:
	dev = "cpu"

device = torch.device(dev)

#torch.cuda.empty_cache()

In [ ]:
use_cuda=False


In [ ]:
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="sshleifer/tinier_bart",
    args=model_args
)

In [ ]:
model.train_model(train_df, eval_data=eval_df)

to_predict = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(eval_df["prefix"].tolist(), eval_df["input_text"].tolist())
]
truth = eval_df["target_text"].tolist()

preds = model.predict(to_predict)
# Saving the predictions if needed
os.makedirs("predictions", exist_ok=True)
type(preds)

  0%|          | 0/240 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/240 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/240 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

list

In [ ]:
eval_df['prefix'] = preds

In [ ]:
eval_df

,input_text,target_text,prefix,Predictions
2,FBI Agent Nicole Scott: You want another crool...,['FBI Agent Nicole Scott: You want another cro...,[ Major oh Shake castleience touchscreen Eas ...,[ Major oh Shake castleience touchscreen Eas ...
14,Deacon: Dies a pacifist or lives a man. Dr. Da...,['Deacon: Dies a pacifist or lives a man. Dr. ...,[Bat differently assembled batch distributpres...,[Bat differently assembled batch distributpres...
15,Deacon: Dies a pacifist or lives a man. Dr. Da...,['Deacon: Dies a pacifist or lives a man. Dr. ...,[ namela landdburing 249 Ideal militiaAppearan...,[ namela landdburing 249 Ideal militiaAppearan...
22,Caroline Channing: I probably feel worse then ...,['Caroline Channing: I probably feel worse the...,[3ib balls Yeah RedsGu treaty Tues commem indi...,[3ib balls Yeah RedsGu treaty Tues commem indi...
36,Tommy Solomon: This Valentine's Day is about m...,"[""Tommy Solomon: This Valentine's Day is about...","[ f Kou[""yr expenditures Earl Order Vladimir s...","[ f Kou[""yr expenditures Earl Order Vladimir s..."
39,Randy: Your chick is really drag. Jimmy Moon: ...,"[""Randy: Your chick is really drag. Jimmy Moon...",[]level balls fabric LCD alarms covenant gravi...,[]level balls fabric LCD alarms covenant gravi...
47,"Sir Richard Cordwell: You're a bastard, frost!...","[""Sir Richard Cordwell: You're a bastard, Fros...",[ibib McK bottleneckbody Substanceapist vivoap...,[ibib McK bottleneckbody Substanceapist vivoap...
50,[Ashley is trying to select a dress for her bi...,['[Ashley is trying to select a dress for her ...,"[, aib3bert spaced medic careful grilled prop...","[, aib3bert spaced medic careful grilled prop..."
51,"Wendy Ward: Adam, do you know what I used to d...","['Wendy Ward: Adam, do you know what I used to...",[ was Publication Publication irritFG Gloss Mo...,[ was Publication Publication irritFG Gloss Mo...
57,Jerry Fletcher: Are you really a hundred? Adam...,['Jerry Fletcher: Are you really a hundred? Ad...,[ a thisgaDragonMagazine Claude lit Spart burg...,[ a thisgaDragonMagazine Claude lit Spart burg...


In [ ]:
#source= :[https://towardsdatascience.com/bart-for-paraphrasing-with-simple-transformers-7c9ea3dfdd8c]

In [ ]:
eval_df.to_csv('/content/gdrive/My Drive/Colab Notebooks/eval_df_1.csv', sep="\t", index=False)